In [ ]:
# -*- coding: utf-8 -*-
"""Retinopathy_on_Colab.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1_BvUiFZlatdaZusk_MYgBF0AdpxXw8-p

This is recent Kaggle Cometition.
Link here: https://www.kaggle.com/c/aptos2019-blindness-detection/overview

You are provided with a large set of retina images taken using fundus photography under a variety of imaging conditions.

A clinician has rated each image for the severity of diabetic retinopathy on a scale of 0 to 4:

0 - No DR

1 - Mild

2 - Moderate

3 - Severe

4 - Proliferative DR

Like any real-world data set, you will encounter noise in both the images and labels. Images may contain artifacts, be out of focus, underexposed, or overexposed. The images were gathered from multiple clinics using a variety of cameras over an extended period of time, which will introduce further variation.

For the purpose of enabling running this over Colab, I have resized the images to 300x300 pixels and provided two numpy files containing all the data. One with images and other with their labels. If you want access to full resulution image you may download from the kaggle page. The total download size is alomost 9 GB.  

**Assignment 9:**
1. Split the data (3662 images) into a training and test set.
2. Make a CNN to predict the labels for the retinal images.
"""

#Force google colab to switch to high ram mode. Run it only once. 
a = []
while(1):
    a.append('1')
#Also change runtime type to enable GPU from menu.

##This block is only for access of files using google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import cv2;
import matplotlib.pyplot as plt
import numpy as np;
import pandas as pd
from random import shuffle;
import cv2;
from random import shuffle;
from tqdm import tqdm;
import tensorflow;
from tensorflow.keras import layers;
from tensorflow.keras import Model;
from tensorflow.keras.optimizers import SGD;
from tensorflow.keras.callbacks import TensorBoard;
IMAGE_SIZE = 300;

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
train_Data_X_File = drive.CreateFile({'id': '1IJRvy-hV3sgEzjJVMnAtieeaASnjClZ1'});
train_Data_Y_File = drive.CreateFile({'id': '1_flYvBAJCp-vP0lYWkAEH3NgkS6Oqa9W'});

#This block takes 4-5 minutes to load the training data.
train_Data_X_File.GetContentFile('train_Data_X.npy');
train_Data_X = np.load('train_Data_X.npy', allow_pickle=True)

train_Data_Y_File.GetContentFile('train_Data_Y.npy');
train_Data_Y = np.load('train_Data_Y.npy', allow_pickle=True)

train_Data_X.shape
#No of training images, First columns is of images and second is of labels

#Show some training images with labels
count = 0;
Num_of_Images = 20;
plt.figure(figsize=(20,20))
label = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR'];
for i in np.random.randint(1000, size = Num_of_Images):
  count = count+1;
  plt.subplot(Num_of_Images/4,4, count);
  plt.imshow(np.reshape(train_Data_X[i],(IMAGE_SIZE,IMAGE_SIZE,3)));
  plt.title(label[int(train_Data_Y[i])]);

#Splitting data into training and testing sets
num_of_Rows = train_Data_X.shape[0];
num_of_columns = train_Data_X.shape[1];
training_X = train_Data_X[:int(np.round(num_of_Rows*0.8))]   #80% data into training
testing_X = train_Data_X[int(np.round(num_of_Rows*0.8)):]     #20% data into testing
training_Y = train_Data_Y[:int(np.round(num_of_Rows*0.8))]   #80% data into training
testing_Y = train_Data_Y[int(np.round(num_of_Rows*0.8)):]     #20% data into testing

# Input images are of size 300x300x3 in which image pixels(300x300) and 3 for the three color channels: R, G, and B
img_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# First convolution extracts 16 filters that are 3x3 Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3 Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3 Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 128 filters that are 3x3 Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Create output layer with a five nodes and softmax activation
output = layers.Dense(5, activation='softmax')(x)

# Create model:
model = Model(img_input, output)

model.summary()

#Compiling the model
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.00005),metrics=['acc']);

# Prepare the data for the training by separating X and Y vectors.
X_train = np.array([i[0] for i in tqdm(training_X)]);
Y_train = np.array([i[0] for i in training_Y]);

X_test = np.array([i[0] for i in tqdm(testing_X)]);
Y_test = np.array([i[0] for i in testing_Y]);

X_train.shape

#Fitting training and testing dataset into model
Model_fit = model.fit(X_train,Y_train, batch_size =64, epochs = 10,verbose=1, validation_data=(X_test, Y_test))

#Evaluating Test Loss and Test accuracy
Marks = model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy percentage:', 100*Marks[1],"%")
print('Test loss perentage:', 100*Marks[0],"%")

import matplotlib.pyplot as plt

#Show some random test images with their predictions as title.
count = 0;
Num_of_Images = 20;
plt.figure(figsize=(20,20))
label = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR'];
for i in np.random.randint(500, size = Num_of_Images):
  count = count+1;
  plt.subplot(Num_of_Images/4,4, count);
  plt.imshow(np.reshape(X_test[i],(IMAGE_SIZE,IMAGE_SIZE,3)));
  P = model.predict(X_test[i].reshape(1,IMAGE_SIZE,IMAGE_SIZE,3)) # Prediction of testing images
  P = np.array(P);
  plt.title(label[int(Y_test[i])]);

#Evaluating accuracy, val_accuracy, loss, val_loss
acc = Model_fit.history['acc']
val_acc = Model_fit.history['val_acc']
loss = Model_fit.history['loss']
val_loss = Model_fit.history['val_loss']
epochs = range(1, len(acc) + 1)

#Plotting Training and Validation accuracy
plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'green', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

#Plotting Training and Validation loss
plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'green', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()
plt.show()